# Model for future Prediction for seismic Facies

In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
import os

# Step 1: Load the pre-trained model
# The model was saved as 'seismic_model.pkl', so I load it from this file
with open('seismic_model.pkl', 'rb') as file:
    model = pickle.load(file)

# Step 2: Load the new test data
# Assuming you have the test data in 'test_data.npy' (features) and 'test_labels.npy' (if available for evaluation)
# Modify these paths accordingly if you're using a different file structure
test_data = np.load('test_data.npy')  # This should be your test features
test_labels = np.load('test_labels.npy')  # Optional: For evaluating the model performance if labels are available

# Step 3: Preprocess the test data
# Depending on how the model was trained, you might need to resize, normalize, or reshape the test data
# Below is an example for resizing and normalizing images:
test_data_resized = []
for i in range(test_data.shape[0]):
    img = test_data[i]  # Assuming each image is 2D
    img_resized = cv2.resize(img, (128, 128))  # Resize the image to match model input dimensions
    img_resized = np.expand_dims(img_resized, axis=2)  # Add a channel dimension
    img_resized = img_resized.astype('float32')
    img_resized = img_resized / np.amax(img_resized)  # Normalize to [0, 1]
    test_data_resized.append(img_resized)

# Convert the list of processed images into a NumPy array
test_data_resized = np.asarray(test_data_resized)

# Step 4: Make predictions using the loaded model
predictions = model.predict(test_data_resized)

# Step 5: Save the predictions to a CSV file
# Convert predictions into a DataFrame for easy saving
predictions_df = pd.DataFrame(predictions)

# Save predictions to CSV (you can change the filename)
predictions_df.to_csv('seismic_predictions.csv', index=False)

# If you want to print out some predictions
print(predictions[:10])  # Show the first 10 predictions

# Step 6: Optionally, evaluate the model on the test set if you have test labels
if 'test_labels' in locals():  # If the test labels are available
    from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

    # For classification tasks (if applicable)
    accuracy = accuracy_score(test_labels, np.argmax(predictions, axis=-1))
    print(f"Accuracy: {accuracy:.4f}")

    # For regression tasks, use metrics like MSE, RMSE, or R^2
    mse = mean_squared_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(test_labels, predictions)
    print(f"MSE: {mse:.4f}, RMSE: {rmse:.4f}, R^2: {r2:.4f}")